In [ ]:
import pandas as pd
import numpy as np
import nltk 
import string
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def load_data():
  data = pd.read_excel('dataset-emosi-twitter-2500.xlsx')
  return data

In [ ]:
teks_df = load_data()

In [ ]:
teks_df.head()

,LABEL,TEKS
0,terkejut,@indomyfess Seokjin\n\nYah pas kenal Bangtan t...
1,jijik,@JaneeeNoLimit Wkwk. Iri dengki dah
2,sedih,Tadi gue lewat jalan yang aga rame orang juala...
3,jijik,@sunrisingbok LWBSKSN yo igual muak
4,terkejut,@moontygf EMG ANJRIT GUA KAGET GWS BGT NAD<U+0...


In [ ]:
df  = pd.DataFrame(teks_df[['LABEL', 'TEKS']])

In [ ]:
df.head(10)

,LABEL,TEKS
0,terkejut,@indomyfess Seokjin\n\nYah pas kenal Bangtan t...
1,jijik,@JaneeeNoLimit Wkwk. Iri dengki dah
2,sedih,Tadi gue lewat jalan yang aga rame orang juala...
3,jijik,@sunrisingbok LWBSKSN yo igual muak
4,terkejut,@moontygf EMG ANJRIT GUA KAGET GWS BGT NAD<U+0...
5,senang,Kelakuanmu itu mencerminkan kelakuan pasanganm...
6,sedih,[USERNAME] [USERNAME] Promosinya tidak terenca...
7,jijik,"@bertanyarl relate banget, kdng ada bbrp sampe..."
8,jijik,spelah dengki OP ni
9,jijik,"jangan iri jangan iri, jangan iri dengki \nJAN..."


In [ ]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt    
df['remove_user'] = np.vectorize(remove_pattern)(df['TEKS'], '@[^\s]+')

In [ ]:
df.head(10)

,LABEL,TEKS,remove_user
0,terkejut,@indomyfess Seokjin\n\nYah pas kenal Bangtan t...,"Seokjin\n\nYah pas kenal Bangtan tiba"" lihat ..."
1,jijik,@JaneeeNoLimit Wkwk. Iri dengki dah,Wkwk. Iri dengki dah
2,sedih,Tadi gue lewat jalan yang aga rame orang juala...,Tadi gue lewat jalan yang aga rame orang juala...
3,jijik,@sunrisingbok LWBSKSN yo igual muak,LWBSKSN yo igual muak
4,terkejut,@moontygf EMG ANJRIT GUA KAGET GWS BGT NAD<U+0...,EMG ANJRIT GUA KAGET GWS BGT NAD<U+0001F64F><...
5,senang,Kelakuanmu itu mencerminkan kelakuan pasanganm...,Kelakuanmu itu mencerminkan kelakuan pasanganm...
6,sedih,[USERNAME] [USERNAME] Promosinya tidak terenca...,[USERNAME] [USERNAME] Promosinya tidak terenca...
7,jijik,"@bertanyarl relate banget, kdng ada bbrp sampe...","relate banget, kdng ada bbrp sampe gue capek ..."
8,jijik,spelah dengki OP ni,spelah dengki OP ni
9,jijik,"jangan iri jangan iri, jangan iri dengki \nJAN...","jangan iri jangan iri, jangan iri dengki \nJAN..."


In [ ]:
def remove(tweet):
     
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
    tweet = re.sub("[u\U0001F600-\U0001F64F]+",'', tweet)
    #remove 
    tweet = re.sub("USERNAME", '', tweet)
    
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    #tweet = re.sub(r'<[\sW]>+', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    return tweet
df['remove_http'] = df['remove_user'].apply(lambda x: remove(x))

In [ ]:
df.head(50)

,LABEL,TEKS,remove_user,remove_http
0,terkejut,@indomyfess Seokjin\n\nYah pas kenal Bangtan t...,"Seokjin\n\nYah pas kenal Bangtan tiba"" lihat ...","Seokjin\n\nYah pas kenal Bangtan tiba"" lihat ..."
1,jijik,@JaneeeNoLimit Wkwk. Iri dengki dah,Wkwk. Iri dengki dah,Wkwk. Iri dengki dah
2,sedih,Tadi gue lewat jalan yang aga rame orang juala...,Tadi gue lewat jalan yang aga rame orang juala...,Tadi ge lewat jalan yang aga rame orang jalan....
3,jijik,@sunrisingbok LWBSKSN yo igual muak,LWBSKSN yo igual muak,LWBSKSN yo igal mak
4,terkejut,@moontygf EMG ANJRIT GUA KAGET GWS BGT NAD<U+0...,EMG ANJRIT GUA KAGET GWS BGT NAD<U+0001F64F><...,EMG ANJRIT GUA KAGET GWS BGT NAD<U+FF><U+FF>
5,senang,Kelakuanmu itu mencerminkan kelakuan pasanganm...,Kelakuanmu itu mencerminkan kelakuan pasanganm...,Kelakanm it mencerminkan kelakan pasanganm nan...
6,sedih,[USERNAME] [USERNAME] Promosinya tidak terenca...,[USERNAME] [USERNAME] Promosinya tidak terenca...,"[] [] Promosinya tidak terencana dengan baik, ..."
7,jijik,"@bertanyarl relate banget, kdng ada bbrp sampe...","relate banget, kdng ada bbrp sampe gue capek ...","relate banget, kdng ada bbrp sampe ge capek n..."
8,jijik,spelah dengki OP ni,spelah dengki OP ni,spelah dengki OP ni
9,jijik,"jangan iri jangan iri, jangan iri dengki \nJAN...","jangan iri jangan iri, jangan iri dengki \nJAN...","jangan iri jangan iri, jangan iri dengki \nJAN..."


In [ ]:
df.sort_values("remove_http", inplace = True)
df.drop_duplicates(subset ="remove_http", keep = 'first', inplace = True)

In [ ]:
df.head(50)

,LABEL,TEKS,remove_user,remove_http
392,terkejut,@wajahsementara @atokprihartono @pieterpitojo ...,Halah <U+0001F92A>\nDiki...,Halah <U+FA>\nDikira say...
95,terkejut,@haikyv @rainboekiss @bindthoughts @bangsatwan...,mw heran tp km memb ofl,mw heran tp km memb ofl
580,jijik,@gulalikoo @greenthanos2 @ZeeTia4 @ButterRockP...,Sebenerny gw dongkol.pengen maki..cm gw ...,Sebenerny gw dongkol.pengen maki..cm gw ...
1907,jijik,@piripiriwonpir @Veenntus @Nachan99718917 @mic...,"Iya wkwk d ruko gw gt jg, prinsip mak bpk...","Iya wkwk d rko gw gt jg, prinsip mak bpk ..."
2374,terkejut,@humasjakfire @DKIJakarta @aniesbaswedan @Ariz...,"Takjub sama diri sendiri,\nKejadian tadi ...","Takjb sama diri sendiri,\nKejadian tadi m..."
2414,terkejut,@wjmilki @hirunkitd @sleuthunch @uraszayas *te...,*tercolek*\n............ Kaget-! https://t...,*tercolek*\n............ Kaget-! https://t...
1593,terkejut,@malestreet09 @Wekmuwekuu @richardeanzzz @d_St...,Innalillahi Wa Innaillaihiroji'un.. semoga...,Innalillahi Wa Innaillaihiroji'n.. semoga ...
1634,terkejut,@tobiramaluv @NaruHina_Indo @txtdarinaruto @sh...,Wah cakep Mbah saya aja cukup bawa gitar s...,Wah cakep Mbah saya aja ckp bawa gitar sam...
106,terkejut,@SRyanto94638978 @__AnakKolong @DKIJakarta <U+...,<U+0001F923><U+0001F923><U+0001F923><U+0001...,<U+F><U+F><U+F><U+F><U+F><U+F><U+F><U+F>lah...
305,senang,@AdamZamri8 @SykrhZlkfly @ASTROARENA <U+0001F6...,<U+0001F60D><U+0001F60D><U+0001F60D>harap m...,<U+FD><U+FD><U+FD>harap mereka menang. Kala...


In [ ]:
  import nltk
  nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('indonesian')
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
#tokenize
from nltk.tokenize import TweetTokenizer
 
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3',"<U+F>","<U+FC>","<U+FD>","<U+FA>","<U+FG>"
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)
 
def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    #remove coma
    tweet = re.sub(r',','',tweet)
    
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
 
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_indonesia and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
 
    return tweets_clean

In [ ]:
pip install Sastrawi

     |████████████████████████████████| 209 kB 5.1 MB/s 


In [ ]:
df['tweet_clean'] = df['remove_http'].apply(lambda x: clean_tweets(x))

In [ ]:
df.head(50)

,LABEL,TEKS,remove_user,remove_http,tweet_clean
392,terkejut,@wajahsementara @atokprihartono @pieterpitojo ...,Halah <U+0001F92A>\nDiki...,Halah <U+FA>\nDikira say...,"[halah, u fa, takjb, git, , u f, plagiat, u f,..."
95,terkejut,@haikyv @rainboekiss @bindthoughts @bangsatwan...,mw heran tp km memb ofl,mw heran tp km memb ofl,"[mw, heran, tp, km, memb, ofl]"
580,jijik,@gulalikoo @greenthanos2 @ZeeTia4 @ButterRockP...,Sebenerny gw dongkol.pengen maki..cm gw ...,Sebenerny gw dongkol.pengen maki..cm gw ...,"[sebenerny, gw, dongkol ken, maki, , cm, gw, t..."
1907,jijik,@piripiriwonpir @Veenntus @Nachan99718917 @mic...,"Iya wkwk d ruko gw gt jg, prinsip mak bpk...","Iya wkwk d rko gw gt jg, prinsip mak bpk ...","[iya, wkwk, d, rko, gw, gt, jg, prinsip, mak, ..."
2374,terkejut,@humasjakfire @DKIJakarta @aniesbaswedan @Ariz...,"Takjub sama diri sendiri,\nKejadian tadi ...","Takjb sama diri sendiri,\nKejadian tadi m...","[takjb, jadi, malem, banget, habis, ambil, dit..."
2414,terkejut,@wjmilki @hirunkitd @sleuthunch @uraszayas *te...,*tercolek*\n............ Kaget-! https://t...,*tercolek*\n............ Kaget-! https://t...,"[colek, , kaget]"
1593,terkejut,@malestreet09 @Wekmuwekuu @richardeanzzz @d_St...,Innalillahi Wa Innaillaihiroji'un.. semoga...,Innalillahi Wa Innaillaihiroji'n.. semoga ...,"[innalillahi, wa, innaillaihiroji n, , moga, l..."
1634,terkejut,@tobiramaluv @NaruHina_Indo @txtdarinaruto @sh...,Wah cakep Mbah saya aja cukup bawa gitar s...,Wah cakep Mbah saya aja ckp bawa gitar sam...,"[cakep, mbah, aja, ckp, bawa, gitar, pamer, ab..."
106,terkejut,@SRyanto94638978 @__AnakKolong @DKIJakarta <U+...,<U+0001F923><U+0001F923><U+0001F923><U+0001...,<U+F><U+F><U+F><U+F><U+F><U+F><U+F><U+F>lah...,"[u f, u f, u f, u f, u f, u f, u f, u f, cma, ..."
305,senang,@AdamZamri8 @SykrhZlkfly @ASTROARENA <U+0001F6...,<U+0001F60D><U+0001F60D><U+0001F60D>harap m...,<U+FD><U+FD><U+FD>harap mereka menang. Kala...,"[u fd, u fd, u fd, harap, menang, kalah, bangg..."


In [ ]:
def remove_punct(text):
    text  = " ".join([char for char in text if char not in string.punctuation])
    return text

In [ ]:
df['Tweet'] = df['tweet_clean'].apply(lambda x: remove_punct(x))

In [ ]:
df

,LABEL,TEKS,remove_user,remove_http,tweet_clean,Tweet
392,terkejut,@wajahsementara @atokprihartono @pieterpitojo ...,Halah <U+0001F92A>\nDiki...,Halah <U+FA>\nDikira say...,"[halah, u fa, takjb, git, , u f, plagiat, u f,...",halah u fa takjb git u f plagiat u f tkang kla...
95,terkejut,@haikyv @rainboekiss @bindthoughts @bangsatwan...,mw heran tp km memb ofl,mw heran tp km memb ofl,"[mw, heran, tp, km, memb, ofl]",mw heran tp km memb ofl
580,jijik,@gulalikoo @greenthanos2 @ZeeTia4 @ButterRockP...,Sebenerny gw dongkol.pengen maki..cm gw ...,Sebenerny gw dongkol.pengen maki..cm gw ...,"[sebenerny, gw, dongkol ken, maki, , cm, gw, t...",sebenerny gw dongkol ken maki cm gw tahan u fd...
1907,jijik,@piripiriwonpir @Veenntus @Nachan99718917 @mic...,"Iya wkwk d ruko gw gt jg, prinsip mak bpk...","Iya wkwk d rko gw gt jg, prinsip mak bpk ...","[iya, wkwk, d, rko, gw, gt, jg, prinsip, mak, ...",iya wkwk d rko gw gt jg prinsip mak bpk gw kal...
2374,terkejut,@humasjakfire @DKIJakarta @aniesbaswedan @Ariz...,"Takjub sama diri sendiri,\nKejadian tadi ...","Takjb sama diri sendiri,\nKejadian tadi m...","[takjb, jadi, malem, banget, habis, ambil, dit...",takjb jadi malem banget habis ambil dit ters p...
...,...,...,...,...,...,...
1880,senang,yang paling aku benci adalah orang yang ingin ...,yang paling aku benci adalah orang yang ingin ...,yang paling ak benci adalah orang yang ingin m...,"[ak, benci, orang, ambil, hak, kebebasank, kai...",ak benci orang ambil hak kebebasank kaito kid
2204,marah,yang paling ditunggu selama ini ya bukber SD S...,yang paling ditunggu selama ini ya bukber SD S...,yang paling ditngg selama ini ya bkber SD SMP....,"[ditngg, ya, bkber, sd, smp, temen, bener, ben...",ditngg ya bkber sd smp temen bener bener ser b...
240,terkejut,yg bikin kaget doi produk lokal itu jco si.. ...,yg bikin kaget doi produk lokal itu jco si.. ...,yg bikin kaget doi prodk lokal it jco si.. sa...,"[yg, bikin, kaget, doi, prodk, lokal, it, jco,...",yg bikin kaget doi prodk lokal it jco si bread...
203,sedih,yg mna rasa diri tu depressed smpai rasa nk su...,yg mna rasa diri tu depressed smpai rasa nk su...,yg mna rasa diri t depressed smpai rasa nk sic...,"[yg, mna, t, depressed, smpai, nk, sicide, t, ...",yg mna t depressed smpai nk sicide t kene dar ...


In [ ]:
df.sort_values("Tweet", inplace = True)

In [ ]:
df

,LABEL,TEKS,remove_user,remove_http,tweet_clean,Tweet
2482,terkejut,"au nya bagus banget, baru mau baca lagi. Tulis...","au nya bagus banget, baru mau baca lagi. Tulis...","a nya bags banget, bar ma baca lagi. Tlisan ny...","[a, nya, bags, banget, bar, ma, baca, tlisan, ...",a nya bags banget bar ma baca tlisan nya canti...
21,sedih,Aa gervan sayang! Jangan lupa buat senyum yah!...,Aa gervan sayang! Jangan lupa buat senyum yah!...,Aa gervan sayang! Jangan lpa bat senym yah! Bi...,"[aa, gervan, sayang, lpa, bat, senym, yah, bia...",aa gervan sayang lpa bat senym yah biar kam ga...
1980,senang,AAAAA BANGGA BANGET SAMA HOONIE<U+0001F62D><U+...,AAAAA BANGGA BANGET SAMA HOONIE<U+0001F62D><U+...,AAAAA BANGGA BANGET SAMA HOONIE<U+FD><U+FD><U+...,"[aaa, bangga, banget, hoonie, u fd, u fd, u fd...",aaa bangga banget hoonie u fd u fd u fd u fd u fd
198,jijik,@S_SiscaJKT48 @Tawan_V Aaaaaaa jangan iri jang...,Aaaaaaa jangan iri jangan iri jangan iri den...,Aaaaaaa jangan iri jangan iri jangan iri den...,"[aaaa, iri, iri, iri, dengki, iri, iri, dengki...",aaaa iri iri iri dengki iri iri dengki u fd u ...
1648,sedih,AAAAAK SUKA BANGET. feel sehabis nonton sama k...,AAAAAK SUKA BANGET. feel sehabis nonton sama k...,AAAAAK SUKA BANGET. feel sehabis nonton sama k...,"[aaak, suka, banget, feel, habis, nonton, kaya...",aaak suka banget feel habis nonton kayak the g...
...,...,...,...,...,...,...
1273,jijik,@bakuwachiturro yo tmb muak muak https://t.co/...,yo tmb muak muak https://t.co/6ZuVbcGy1Q,yo tmb mak mak https://t.co/ZVbcGyQ,"[yo, tmb, mak, mak]",yo tmb mak mak
417,senang,1. Yoon Jisung Aslinya lebih ganteng drpda fot...,1. Yoon Jisung Aslinya lebih ganteng drpda fot...,. Yoon Jisng Aslinya lebih ganteng drpda foto ...,"[yoon, jisng, asli, ganteng, drpda, foto, eman...",yoon jisng asli ganteng drpda foto emang peson...
658,sedih,[USERNAME] [USERNAME] Berarti Yusufk09 seringn...,[USERNAME] [USERNAME] Berarti Yusufk09 seringn...,[] [] Berarti Ysfk seringnya nonton penyanyi d...,"[ysfk, nonton, nyanyi, dangdt, yg, cma, jal, g...",ysfk nonton nyanyi dangdt yg cma jal goyang at...
513,senang,[USERNAME] [USERNAME] [USERNAME] [USERNAME] [U...,[USERNAME] [USERNAME] [USERNAME] [USERNAME] [U...,[] [] [] [] [] [] [] [] Zon jangan lihat bajny...,"[zon, lihat, bajnya, sajasekali-sekali, pegang...",zon lihat bajnya sajasekali-sekali pegang brng...


In [ ]:
df.drop(df.columns[[1,2,3,4]], axis = 1, inplace = True)

In [ ]:
df

,LABEL,Tweet
2482,terkejut,a nya bags banget bar ma baca tlisan nya canti...
21,sedih,aa gervan sayang lpa bat senym yah biar kam ga...
1980,senang,aaa bangga banget hoonie u fd u fd u fd u fd u fd
198,jijik,aaaa iri iri iri dengki iri iri dengki u fd u ...
1648,sedih,aaak suka banget feel habis nonton kayak the g...
...,...,...
1273,jijik,yo tmb mak mak
417,senang,yoon jisng asli ganteng drpda foto emang peson...
658,sedih,ysfk nonton nyanyi dangdt yg cma jal goyang at...
513,senang,zon lihat bajnya sajasekali-sekali pegang brng...


In [ ]:
df.drop_duplicates(subset ="Tweet", keep = 'first', inplace = True)

In [ ]:
df

,LABEL,Tweet
2482,terkejut,a nya bags banget bar ma baca tlisan nya canti...
21,sedih,aa gervan sayang lpa bat senym yah biar kam ga...
1980,senang,aaa bangga banget hoonie u fd u fd u fd u fd u fd
198,jijik,aaaa iri iri iri dengki iri iri dengki u fd u ...
1648,sedih,aaak suka banget feel habis nonton kayak the g...
...,...,...
1635,jijik,yo tmb mak mak
417,senang,yoon jisng asli ganteng drpda foto emang peson...
658,sedih,ysfk nonton nyanyi dangdt yg cma jal goyang at...
513,senang,zon lihat bajnya sajasekali-sekali pegang brng...


In [ ]:
df.to_excel('output.xlsx',encoding='utf8', index=False)
df

,LABEL,Tweet
21,sedih,aa gervan sayang lupa senyum yah biar ganteng ...
1980,senang,aaa bangga banget hoonie u fd u fd u fd u fd u fd
198,jijik,aaaa iri iri iri dengki iri iri dengki u fd u ...
1648,sedih,aaak suka banget feel habis nonton kayak the g...
2435,sedih,abah tinggal cancer lidah amp of course rokoki...
...,...,...
2097,jijik,yo mas muak mauk
1273,jijik,yo tmb muak muak
417,senang,yoon jisung asli ganteng drpda foto emang peso...
1815,takut,yuk mari dukung omnibus law ruu cipta kerja te...
